<a href="https://colab.research.google.com/github/mituchowdhury1/data-mining/blob/main/ASSIGNMENT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Name: Mitu
### ID: 2125051019

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load the Dataset
ratings = pd.read_csv('/content/drive/MyDrive/Data mining/ratings.csv')
movies = pd.read_csv('/content/drive/MyDrive/Data mining/movies.csv')

# Step 2: Create User-Movie Rating Matrix
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')
user_movie_matrix_filled = user_movie_matrix.fillna(0)

# Step 3: Compute Movie Similarity Matrix
movie_similarity = cosine_similarity(user_movie_matrix_filled.T)
movie_similarity_df = pd.DataFrame(movie_similarity,
                                   index=user_movie_matrix.columns,
                                   columns=user_movie_matrix.columns)

# Step 4: Find Movies Similar to a Given Movie
def get_similar_movies(movie_id, top_n=5):
    if movie_id not in movie_similarity_df.columns:
        print("Movie ID not found.")
        return pd.DataFrame()

    similar_scores = movie_similarity_df[movie_id]
    similar_movies = similar_scores.sort_values(ascending=False).drop(movie_id)
    top_similar = similar_movies.head(top_n)

    return movies[movies['movieId'].isin(top_similar.index)]

# Step 5: Recommend Movies for a User
def recommend_movies_for_user(user_id, top_n=5):
    if user_id not in user_movie_matrix.index:
        print("User ID not found.")
        return pd.DataFrame()

    # Movies rated by the user
    user_ratings = user_movie_matrix.loc[user_id]
    rated_movies = user_ratings[user_ratings.notna()].index.tolist()

    # Top 3 highest-rated movies
    top_rated_movies = user_ratings.sort_values(ascending=False).head(3).index.tolist()

    similar_movies_list = []  # Fixed: use list instead of append

    for movie_id in top_rated_movies:
        similar = movie_similarity_df[movie_id]
        similar = similar.drop(movie_id)
        similar_movies_list.append(similar)

    # Combine all similar movies
    similar_movies = pd.concat(similar_movies_list)

    # Remove already rated movies
    similar_movies = similar_movies[~similar_movies.index.isin(rated_movies)]

    # Group by movie ID and take mean similarity
    similar_movies = similar_movies.groupby(similar_movies.index).mean()

    # Merge with average ratings
    avg_movie_ratings = ratings.groupby('movieId')['rating'].mean()
    recommendations = similar_movies.to_frame('similarity').join(avg_movie_ratings, how='left')

    # Calculate final score
    recommendations['final_score'] = recommendations['similarity'] * 0.7 + recommendations['rating'] * 0.3

    top_recommendations = recommendations.sort_values('final_score', ascending=False).head(top_n)

    return movies[movies['movieId'].isin(top_recommendations.index)]

# ============================
# Example Usage
# ============================

# 1. Find movies similar to Toy Story
print("\n🎬 Movies similar to 'Toy Story (1995)':")
toy_story_id = movies[movies['title'] == 'Toy Story (1995)']['movieId'].values[0]
similar_to_toy_story = get_similar_movies(toy_story_id, top_n=5)
print(similar_to_toy_story[['title', 'genres']])

# 2. Recommend movies for userId = 1
print("\n👤 Recommended movies for User 1:")
recommendations = recommend_movies_for_user(1, top_n=5)
print(recommendations[['title', 'genres']])



🎬 Movies similar to 'Toy Story (1995)':
                                          title  \
224   Star Wars: Episode IV - A New Hope (1977)   
314                         Forrest Gump (1994)   
418                        Jurassic Park (1993)   
615        Independence Day (a.k.a. ID4) (1996)   
2355                         Toy Story 2 (1999)   

                                           genres  
224                       Action|Adventure|Sci-Fi  
314                      Comedy|Drama|Romance|War  
418              Action|Adventure|Sci-Fi|Thriller  
615              Action|Adventure|Sci-Fi|Thriller  
2355  Adventure|Animation|Children|Comedy|Fantasy  

👤 Recommended movies for User 1:
                                 title                 genres
277   Shawshank Redemption, The (1994)            Crime|Drama
659              Godfather, The (1972)            Crime|Drama
3110        Reform School Girls (1986)           Action|Drama
3807                       Rain (2001)          Drama|Roma